In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

def DEUTERONSF(ibeam, istruc, inuke, itm, iht, ndrv, v, xc, Fd):
    # Decompose 'inuke' into components and determine the smearing model
    ismear = split_nuke(inuke)
    
    if ismear <= 1:
        # Isoscalar p+n calculations
        Fd = strfn(ibeam, istruc, itm, iht, ndrv, v, xc)
        if ismear == 1:
            # Apply density model corrections
            dmc = gomez(1.0, v[0])
            Fd /= dmc
    elif 2 <= ismear <= 9:
        # Nuclear smearing model
        varr = [0, v[1]]  # Q^2 value
        FNarr = []
        for ix in range(1, nx+1):
            x = float(ix) / 100.0 - 0.0001
            varr[0] = x  # x value
            FNarr.append(strfn(ibeam, istruc, itm, iht, ndrv, varr, xc))
        # Smear the nucleon structure function
        Fd = SMEARF2(ibeam, istruc, inuke, itm, iht, ndrv, v, xc, FNarr)
    else:
        print(f'ERROR(DEUTERONSF): inuke out of range = {inuke}')

    return Fd

# Predefined constants from SciPy
pi = constants.pi  # Pi
hc = constants.h * constants.c / constants.eV * 1e6  # Planck constant times speed of light in MeV fm (h * c / eV * 1e6 to convert to MeV fm)

# Nucleon mass in MeV/c^2 (average might differ slightly from precise proton/neutron masses)
mN = 938.91897  # This specific value might need to be defined manually if exact match is necessary

# Calculated or manually defined constants
mNGeV = mN / 1e3  # Nucleon mass in GeV/c^2
mD = 2 * mN - 2.224575  # Deuteron mass in MeV/c^2 calculated from nucleon mass and binding energy

print(f"pi: {pi}")
print(f"hc (Planck constant * speed of light in MeV fm): {hc}")
print(f"mN (Nucleon mass in MeV/c^2): {mN}")
print(f"mNGeV (Nucleon mass in GeV/c^2): {mNGeV}")
print(f"mD (Deuteron mass in MeV/c^2): {mD}")

import numpy as np

def compute_smearing_model(v, inuke, mN, mD, mNGeV):
    """
    Computes smearing model parameters.
    
    Parameters:
    - v: A list or array containing at least two elements, with v[0] being 'x' and v[1] being 'Q2'.
    - inuke: Integer representing nuclear corrections and other parameters.
    - mN: Mass of the nucleon in MeV/c^2.
    - mD: Mass of the deuteron in MeV/c^2.
    - mNGeV: Mass of the nucleon in GeV/c^2.
    
    Returns:
    - A dictionary containing the computed 'x', 'xD', 'Q2', and 'gamma'.
    """
    
    # Assuming split_nuke is a function that returns a tuple of parameters based on 'inuke'
    ilam, ishad, istrg, ioff, iwfn, ismear = split_nuke(inuke)
    
    # Value of x
    x = v[0]
    
    # Modified Bjorken variable for the deuteron, adjusted by the ratio of nucleon mass to deuteron mass
    xD = x * (mN / mD)
    
    # Value of Q2
    Q2 = v[1]
    
    # Calculation of gamma based on 'ilam'
    if ilam >= 1:
        # Bjorken limit
        gamma = 1.0
    else:
        # Finite Q^2
        gamma = np.sqrt(1.0 + 4 * x**2 * mNGeV**2 / Q2)
    
    return {'x': x, 'xD': xD, 'Q2': Q2, 'gamma': gamma}
# Initialization
yDmax = 1.0
yDmin = xD  # Assuming xD is defined
FFD = 0.0
FFDoff = 0.0

# Placeholder functions for those called in the Fortran code
def PHI_INT2D(yD, gamma, mode, ismear, iwfn):
    # Placeholder for the actual implementation
    return 0.0

def strfn(ibeam, istruc, itm, iht, ndrv, v, xc):
    # Placeholder for the actual implementation
    return 0.0

def set_offshell_on(state):
    # Placeholder for the actual implementation
    pass

def fsupdf(arg1, x, s, u, d, ub, db, sb, cb, bb, glue):
    # Placeholder for the actual implementation
    pass

############################################################################################ THIS LOOP NEEDS WORK ############################################################################################
# Main loop
for I in range(NTERMS):  # Assuming NTERMS is defined
    yD = 0.5 * (yDmax - yDmin) * XI[I] + 0.5 * (yDmax + yDmin)
    v[0] = xD / yD  # Adjusting the index for Python

    # Conditional logic
    if istruc == 0:
        # Example conditional block
        fy_diag = PHI_INT2D(yD, gamma, 0, ismear, iwfn)
        FFN = strfn(ibeam, 0, itm, iht, ndrv, v, xc)
        fy_off = PHI_INT2D(yD, gamma, 1, ismear, iwfn)
        F2N = strfn(ibeam, 2, itm, iht, ndrv, v, xc)
        
        if ioff == 1 or (5 <= ioff <= 9):
            foff_diag = PHI_INT2D(yD, gamma, 10, ismear, iwfn)
            foff_off = PHI_INT2D(yD, gamma, 11, ismear, iwfn)
            set_offshell_on(True)
            FFNoff = strfn(ibeam, 0, itm, iht, ndrv, v, xc)
            F2Noff = strfn(ibeam, 2, itm, iht, ndrv, v, xc)
            set_offshell_on(False)
    # Repeat for other conditions as in the Fortran code

    # Additional conditions and computations as per the Fortran code...

    # Example of final computation within the loop
    if v[0] < 0.995:
        weight = 0.5 * (yDmax - yDmin) * WI[I]
        # Further computations based on the condition...
#############################################################################################################################################################################################################
        
def split_nuke(inuke):
    ilam = inuke // 100000  # treatment of TMC scale Lambda (A)
    ishad = (inuke - ilam * 100000) // 10000  # shadowing corrections (B)
    istrg = (inuke - ilam * 100000 - ishad * 10000) // 1000  # strength of off-shell corrections (C)
    ioff = (inuke - ilam * 100000 - ishad * 10000 - istrg * 1000) // 100  # off-shell corrections (D)
    iwfn = (inuke - ilam * 100000 - ishad * 10000 - istrg * 1000 - ioff * 100) // 10  # nuclear wave function (E)
    ismear = inuke - ilam * 100000 - ishad * 10000 - istrg * 1000 - ioff * 100 - iwfn * 10  # convolution formula or deuteron correction model (F)

    return ilam, ishad, istrg, ioff, iwfn, ismear

# Example usage
inuke = 123456  # Example input
ilam, ishad, istrg, ioff, iwfn, ismear = split_nuke(inuke)
print("ilam:", ilam, "ishad:", ishad, "istrg:", istrg, "ioff:", ioff, "iwfn:", iwfn, "ismear:", ismear)
# Coefficients p1 to p7: These are the parameters of the polynomial fit to the 
# F2D/F2N data, as determined by Gomez et al. The fit is expressed as a polynomial function of the variable 
# x
# x, which is typically the Bjorken scaling variable in DIS experiments.
# The Polynomial Fit (THEORY): The variable THEORY is calculated as a polynomial in 
# x
# x up to the fifth power, using the coefficients p1 to p6.
# Deuteron Correction Factor (deucor): This factor accounts for the behavior of the deuteron structure function at low 
# x
# x values. The term deufac is calculated, which is then used to compute deucor as 
# 1
# −
# exp
# ⁡
# (
# −
# deufac
# )
# 1−exp(−deufac). The deufac value is capped at 20 to avoid numerical instabilities.
# Correction Application: The THEORY value is divided by deucor to apply the deuteron correction.
# Output (gomez): The function returns the ratio of an input function value f to the corrected theoretical value. This can be used to scale the input function by the deuteron correction factor derived from the Gomez et al. data.

def gomez(f, x):
    # Coefficients from the polynomial fit
    p1, p2, p3, p4, p5, p6, p7 = 1.0164, -0.047762, -0.13354, 2.35303, 0.22719, -1.2906, 5.6075

    # Calculate the polynomial fit (THEORY) and the deuteron correction factor (deucor)
    theory = p1 + p2*x + p3*x**2 + p4*x**3 + p5*x**4 + p6*x**5
    deufac = p7 * (1.0/x - 1.0)
    deufac = min(deufac, 20)  # Cap deufac at 20
    deucor = 1.0 - np.exp(-deufac)
    
    # Apply the deuteron correction
    theory_corrected = theory / deucor
    
    # Return the ratio of the input function to the corrected theory
    return f / theory_corrected

def OFF_MST(x):
    # Parameters specific to the MST (Melnitchouk, Schreiber, Thomas) model. They define the shape of the off-shell correction as a function of nucleon scaling variable x.
    aoff = [-0.014, 3.0, 20.0, 1.067, 1.5, 18.0]

    # Apply correction only if x is greater than 0.16
    if x <= 0.16:
        return 0.0

    # Calculate the off-shell correction
    off_mst = aoff[0] * (1.0 + aoff[1] * x**aoff[2]) * (1.0 - (aoff[3] - x**aoff[4])**aoff[5])

    # This function returns the calculated off-shell correction factor, which is the ratio of the off-shell correction to the deuteron structure function (F2d)
    return off_mst

def DEL_SHAD(x, Q2):
    if x >= 0.3:
        return 0.0  # Parametrization valid for x <~ 0.3

    # VMD (Q2 in GeV^2)
    p1 = -0.084 * Q2 ** (-0.71)
    p2 = 2.18 + 3.11 * np.exp(-0.689 * Q2)
    DEL_V = p1 * x ** 0.032 * (0.3 - x) ** 1.375 * np.exp(-p2 * x) # Accounts for the Vector Meson Dominance component of shadowing, which is a Q2-dependent effect.

    # Pomeron-exchange
    p1 = -0.0213
    p2 = -0.0883
    p3 = 1.45
    p4 = 18.63
    DEL_P = p1 * x ** p2 * (0.3 - x) ** p3 * np.exp(-p4 * x) # Represents the contribution from Pomeron exchange to shadowing, significant at low x.

    # Meson-exchange (antishadowing)
    p1 = 0.00308
    p2 = 0.103
    p3 = -0.108
    p4 = 7.15
    DEL_M = p1 * x ** p2 * (0.3 - x) ** p3 * np.exp(-p4 * x) # Represents the antishadowing effect due to meson exchange, typically relevant at higher x values within the shadowing region.

    DEL_SHAD = DEL_V + DEL_P + DEL_M # Overall shadowing correction factor. delta(shadow)F2d/F2d

    return DEL_SHAD

# This function constructs a filename based on various input parameters related to nuclear smearing, wave function models, and off-shell corrections.
# It takes an initial string outfile, appends different substrings based on the input parameters ismear, iwfn, and ioff, and returns the modified filename along with its length outlen.
def smearfile(ismear, iwfn, ioff):
    outfile = 'phi.'
    
    # Handling smearing model
    if ismear == 3:
        outfile += 'wba'
    elif ismear == 4:
        outfile += 'wbarel'
    elif ismear == 5:
        outfile += 'aqv'
    elif ismear == 6:
        outfile += 'aqvc'
    elif ismear not in [0, 1]:
        outfile += '_smear??'
    
    # Handling wave function model
    if iwfn == 0:
        outfile += '_paris'
    elif iwfn == 1:
        outfile += '_AV18'
    elif iwfn == 2:
        outfile += '_CDBONN'
    elif iwfn == 3:
        outfile += '_WJC1'
    elif iwfn == 4:
        outfile += '_WJC2'
    else:
        outfile += '_wfn??'
    
    # Handling off-shell corrections
    if ioff == 1:
        outfile += '_KPoff'
    elif ioff != 0:
        outfile += '_offsh??'
    
    outlen = len(outfile)
    
    return outfile, outlen

# Example usage
ismear, iwfn, ioff = 3, 1, 1
outfile, outlen = smearfile(ismear, iwfn, ioff)
print(f"Modified filename: {outfile} (Length: {outlen})")


